In [2]:
import pandas as pd  #bibioteca responsável para o tratamento e limpeza dos dados
import numpy as np #biblioteca utilizada para o tratamento eficiente de dados numéricos
import datetime  #biblioteca utilizada para trabalhar com datas
from matplotlib import pyplot as plt  #plotar os gráficos
import seaborn as sns #plot de gráficos
from pandas import DataFrame

In [3]:
df_coracao=pd.read_csv('doencas_coracao.csv')

In [4]:
df_coracao.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
df_coracao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [6]:
df_coracao.shape

(299, 13)

**Qual era a idade (age) média dos pacientes que faleceram(DEATH_EVENT=1)?**

In [10]:
df_Death = df_coracao[['age','DEATH_EVENT']]
df_Death.query('DEATH_EVENT == 1').describe()

,age,DEATH_EVENT
count,96.000000,96.0
mean,65.215281,1.0
std,13.214556,0.0
min,42.000000,1.0
25%,55.000000,1.0
50%,65.000000,1.0
75%,75.000000,1.0
max,95.000000,1.0


**Dentre os pacientes que sobreviveram (DEATH_EVENT=1), quantos são do sexo feminino (sex=0)?**

In [11]:
df_Gen = df_coracao[['sex','DEATH_EVENT']]
df_Gen.query('DEATH_EVENT == 1').groupby('sex').describe()


DEATH_EVENT                                   
          count mean  std  min  25%  50%  75%  max
sex                                               
0          34.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
1          62.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

**Definindo as entradas e saídas do nosso projeto, conforme enunciado**

In [13]:
dfEntrada = DataFrame(df_coracao,columns=['sex','age','diabetes','anaemia','high_blood_pressure','creatinine_phosphokinase','smoking','ejection_fraction','platelets','serum_sodium','serum_creatinine'])
dfSaida = DataFrame(df_coracao,columns=['DEATH_EVENT'])

**Normalização das entradas**

In [14]:
from sklearn.preprocessing import MinMaxScaler
normaliza = MinMaxScaler() #objeto para a normalizaçã
entradas_normalizadas=normaliza.fit_transform(dfEntrada)


**Divisão entre treino e teste**

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(entradas_normalizadas, dfSaida,
test_size=0.30,random_state=42)
x_train.shape

(209, 11)

**Algoritmo KNN**

In [17]:
#contruindo o modelo para cliassificação com KNN com K=5
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
clf_KNN = KNeighborsClassifier(n_neighbors=5)

#treinando o modelo
clf_KNN.fit(x_train, y_train) 

y_previsto = clf_KNN.predict(x_test) 
acuracia = accuracy_score(y_test, y_previsto)
print('Acurácia da àrvore de Decisão: ',acuracia)



Acurácia da àrvore de Decisão:  0.5777777777777777


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


**Algoritmo Árvore de Decisão**

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
clf_arvore = DecisionTreeClassifier(random_state=1) 
clf_arvore.fit(x_train, y_train) 

#realiza a previsão com os dados
y_previsto = clf_arvore.predict(x_test) 
acuracia = accuracy_score(y_test, y_previsto)
print('Acurácia da àrvore de Decisão: ',acuracia)

Acurácia da àrvore de Decisão:  0.5888888888888889


**Algoritmo Rede MLP**

In [19]:
#MLP como classificador
from sklearn.neural_network import MLPClassifier

#define a configuração da rede
clf_mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 10), random_state=1)

#realiza o fit do modelo
clf_mlp.fit(x_train,y_train.values.ravel())

#realiza a previsão
y_pred_mlp=clf_mlp.predict(x_test)

print(classification_report(y_test,y_pred_mlp))


              precision    recall  f1-score   support

           0       0.69      0.79      0.74        53
           1       0.62      0.49      0.55        37

    accuracy                           0.67        90
   macro avg       0.65      0.64      0.64        90
weighted avg       0.66      0.67      0.66        90



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


**Concorrência - tarefa a ser realizada pelas threads**

In [20]:
#identificando a quantidade de núcleos disponíveis para o sistema
import multiprocessing
multiprocessing.cpu_count() #conta a quantidade de núcleos disponíveis no sistema 
#processamento sequencial
import threading  #módulo para a construção de threads
import urllib.request  #módulo para a requisição de url
import time  #módulo para tratar tempo
import random


In [21]:
def contador():
  x = 1000000
  while x > 0:
    x -=1


In [22]:
def imple_sequencial():
  contador()
  contador()
  

**sequencial**

In [23]:
t0 = time.time()
imple_sequencial()
t1 = time.time()
totalTime = t1 - t0
print("Tempo total de execução {}".format(totalTime))

Tempo total de execução 0.12084436416625977


**Concorrente**

In [24]:
import threading
import time
import random

def imple_concorrente():
  thread_1 = threading.Thread(target=contador)
  thread_2 = threading.Thread(target=contador)

t0 = time.time()

threads = [] #lista vazia que vai conter todas as threads criadas

#cria das 10 threads, cada uma delas será responsável por realizar o download
for i in range(1):
  thread = threading.Thread(target=imple_concorrente, args=(i,))
  threads.append(thread)
  thread.start()
  
#garante que as execuções foram finalizadas
for i in threads:
  i.join()

#calcula o tempo de execução
t1 = time.time()
totalTime = t1 - t0
print("Tempo total de execução {}".format(totalTime))

Tempo total de execução 0.007271289825439453


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: imple_concorrente() takes 0 positional arguments but 1 was given



**Concorrente com tempo randômico**

In [25]:
import threading
import time
import random

def contador1():
  x = 100
  while x > 0:
    x -=1
    time.sleep(random.randint(1,20))

def imple_concorrente():
  thread_1 = threading.Thread(target=contador1)
  thread_2 = threading.Thread(target=contador1)

t0 = time.time()

threads = [] #lista vazia que vai conter todas as threads criadas

#cria das 10 threads, cada uma delas será responsável por realizar a contagem
for i in range(1):
  thread = threading.Thread(target=imple_concorrente, args=(i,))
  threads.append(thread)
  thread.start()
  
#garante que as execuções foram finalizadas
for i in threads:
  i.join()

#calcula o tempo de execução
t1 = time.time()
totalTime = t1 - t0
print("Tempo total de execução {}".format(totalTime))

Tempo total de execução 0.0026133060455322266


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: imple_concorrente() takes 0 positional arguments but 1 was given



**Questões de programação reativa**

In [26]:
!pip install rx

     |████████████████████████████████| 204kB 2.8MB/s 


In [27]:
import rx
import rx.operators as ops
source = rx.from_iterable([5,4,3,2,1]) #streaming
disposable = source.pipe(
    #ops.map(lambda i:i-1),
    ops.filter(lambda i:i%2==0),
).subscribe(
    on_next = lambda i: print("on_next: {}".format(i)),
    on_completed = lambda: print("on_completed"),
    on_error = lambda e:print("on_error: {}".format(e))
) #inscrição do observer

#utilizar como base a implementação presente na primeira aula sobre programação
#reativa


on_next: 4
on_next: 2
on_completed


**Questão 10**

In [28]:
import rx
import rx.operators as ops
source = rx.from_iterable([5,4,3,2,1]) #streaming
disposable = source.pipe(
    ops.map(lambda i:i if i<3 else 0),
    ops.filter(lambda i: i>0),
).subscribe(
    on_next = lambda i: print("on_next: {}".format(i)),
    on_completed = lambda: print("on_completed"),
    on_error = lambda e:print("on_error: {}".format(e))
) #inscrição do observer

#utilizar como base a implementação presente na primeira aula sobre programação
#reativa

on_next: 2
on_next: 1
on_completed


**Questão 11**

In [29]:
import rx
import rx.operators as ops
source = rx.from_iterable([5,4,3,"2",1]) #streaming
disposable = source.pipe(
    ops.filter(lambda i: i%2==0),
).subscribe(
    on_next = lambda i: print("on_next: {}".format(i)),
    on_completed = lambda: print("on_completed"),
    on_error = lambda e:print("on_error: {}".format(e))
) #inscrição do observer

#utilizar como base a implementação presente na primeira aula sobre programação
#reativa

on_next: 4
on_error: not all arguments converted during string formatting
